# REBOUND and NEMO

Here we compare a short leapfrog integration on 100 stars with different integrators.  First ensure NEMO is present with the **nemo** command

If not, you'll need to enable it in this shell with

      source nemo_start.sh

in the directory where NEMO was installed. 

If you started this notebook in a shell where NEMO was present, you should not need to re-enable it. NEMO should be present.


In [1]:
nemo

NEMO:        /home/teuben/NEMO/nemo  - Version:4.4.2
git:         Branch:master     Counter:11651      Date: 2024/01/20_09:57:25
python:      /home/teuben/anaconda3/bin/python  - Python 3.10.9
OS_release:  Linux Description:	Ubuntu 22.04.3 LTS


If REBOUND and the associated NEMO tools are not installed, install them with **mknemo rebound** :

In [2]:
# mknemo rebound

We test this with the **reb_integrate** command:

In [3]:
reb_integrate help=h


in               : input snapshot [???]
out              : output snapshot [???]
tstop            : Set, or let go forever [10]
dtout            : Optional interval to save simulation [1]
dt               : Integration Step [0.01]
eps              : Gravitational Softening Length [0.01]
integrator       : rebound integrator [leapfrog]
gravity          : none, basic, compensated or tree [basic]
tolerance        : tree opening angle - in radians? [0.25]
box              : Optional Simulation Box size - in terms or r0 []
G                : Gravitational Constant [1]
server           : Use web based server on this port (if > 0) [-1]
VERSION          : 17-jan-2024 PJT [0.3]


## Integrating 

We will integrate 100 particles, initialized with **mkplummer**, with a  direct N-body integrator. The **directcode** and **hackcode1** are two NEMO integrators that use a leapfrog integrator that can do this. They should also agree, and we will compare it with **reb_integrate**, the REBOUND integrator compiled with NEMO.

We will take 8 steps with or without dumping every step. Pick to use n=1 output, or n=8 outputs, to check if this is a Heisenbug (it isn't, they both gave the same result)

In [4]:
n=1

In [5]:
n=8

We compare the two integrators
* directcode:      NEMO's N^2 integrator
* hackcode1:       NEMO's tree code, but with tol=0 it will become an N^2 integrator
* reb_integrate:   REBOUND

after this use the **snapcmp** program to see how they diverge

In [6]:
rm -f p100.*
mkplummer p100.a 100 seed=123
directcode p100.a p100.b tstop=1 freq=8 freqout=$n eps=0.05 > p100.b.log
s2reb p100.a p100.c
reb_integrate p100.c p100.d tstop=1 dt=1/8 dtout=1/$n eps=0.05
reb2s p100.d p100.e
snapcmp p100.b p100.e log=t

hackcode1 p100.a p100.f tstop=1 freq=8 freqout=$n eps=0.05 tol=0 > p100.f.log
snapcmp p100.b p100.f log=t


### nemo Debug Info: Read snapshot N=100 at t=0
### nemo Debug Info: Integrator:     4
### nemo Debug Info: Gravity Solver: 1
N_tot=  100       t=  0.000000  dt=  0.125000  cpu=  0.000000 [s]  
### nemo Debug Info: SimulationArchive `p100.d` with 9 snapshots

Warning! You have to reset function pointers after creating a reb_simulation struct with a binary file.
### nemo Debug Info: Snapshot 0 at time 0 with nbody 100

Warning! You have to reset function pointers after creating a reb_simulation struct with a binary file.
### nemo Debug Info: Snapshot 1 at time 0.125 with nbody 100

Warning! You have to reset function pointers after creating a reb_simulation struct with a binary file.
### nemo Debug Info: Snapshot 2 at time 0.25 with nbody 100

Warning! You have to reset function pointers after creating a reb_simulation struct with a binary file.
### nemo Debug Info: Snapshot 3 at time 0.375 with nbody 100

Warning! You have to reset function pointers after creating a reb_simulation stru

The first set of comparisons have a slow divergent drift (note a logarithmic scale), but the directcode and hackcode1 compare well on the machine level (16 digits) as one would expect

## Plotting

Plotting in a bash kernel with NEMO is a bit hard, there is to my knowledge no embedded plotting possible. Instead we use the yapp_pgplot interface and save output as a PNG file, and load this in the next markdown cell. So first we plot the initial conditions, and then compare visually the NEMO and REBOUND results at t=1

In [7]:
snapplot p100.a psize=0.1 yapp=p100.a.png/png
snapplot p100.b psize=0.1 yapp=p100.b.png/png times=1.0
snapplot p100.e psize=0.1 yapp=p100.e.png/png times=1.0

## Initial conditions
![initial](p100.a.png)

## directcode at t=1.0
![directcode t=1](p100.b.png)

## reb_integrate at t=1.0
![reb_integrate t=1](p100.e.png)

